In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## क्लायंट तयार करणे

या उदाहरणात, आपण [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) चा वापर LLM साठी करू. 

`model` म्हणून `gpt-4o-mini` परिभाषित केले आहे. GitHub Models मार्केटप्लेसवर उपलब्ध असलेल्या इतर मॉडेल्समध्ये बदल करून वेगवेगळे परिणाम पाहण्याचा प्रयत्न करा. 

एक जलद चाचणी म्हणून, आपण एक साधा प्रश्न चालवू - `What is the capital of France`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## फंक्शन्सची व्याख्या

या उदाहरणात, आपण एजंटला एका साधनाचा वापर करण्याची परवानगी देऊ, जे उपलब्ध सुट्टीच्या ठिकाणांची यादी आणि त्यांची उपलब्धता असलेली फंक्शन आहे.

तुम्ही असे समजू शकता की हे असे एक प्रकरण आहे जिथे प्रवास एजंटकडे प्रवास डेटाबेसचा प्रवेश असू शकतो.

या नमुन्यातून जात असताना, एजंट कॉल करू शकणाऱ्या नवीन फंक्शन्स आणि साधने परिभाषित करण्याचा प्रयत्न करा.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## फंक्शन टूलची व्याख्या  
एजंटला `vacation_destinations` चा `FunctionTool` म्हणून वापरायचा असल्यास, आपल्याला त्याची व्याख्या करावी लागेल.  

आपण टूलचे वर्णन देखील देऊ, जे एजंटला हे ओळखण्यात मदत करते की वापरकर्त्याने विनंती केलेल्या कार्याशी संबंधित त्या टूलचा उपयोग कशासाठी केला जातो.  


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## एजंटची व्याख्या

आता आपण खालील कोडमध्ये एजंट तयार करू शकतो. आम्ही `system_message` परिभाषित करतो जेणेकरून एजंटला वापरकर्त्यांना सुट्टीसाठी गंतव्यस्थाने शोधण्यात मदत करण्याच्या सूचना दिल्या जाऊ शकतील.

आम्ही `reflect_on_tool_use` पॅरामीटर true वर सेट करतो. यामुळे LLM ला टूल कॉलच्या प्रतिसादाचा उपयोग करून तो नैसर्गिक भाषेत पाठवण्याची परवानगी मिळते.

तुम्ही हा पॅरामीटर false वर सेट करू शकता जेणेकरून फरक पाहता येईल.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## एजंट चालवणे

आता आपण एजंट चालवू शकतो, ज्यामध्ये सुरुवातीचा वापरकर्ता संदेश टोकियोला प्रवास करण्याची विनंती करतो.

तुम्ही या शहराच्या गंतव्यस्थानात बदल करू शकता आणि पाहू शकता की एजंट त्या शहराच्या उपलब्धतेला कसा प्रतिसाद देतो.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) चा वापर करून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर केल्यामुळे उद्भवणाऱ्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
